In [1]:
import os
import re
import numpy as np
import matplotlib.pyplot as plt

In [3]:
l = 'au9-xyz-force.xyz'
a = re.match(r"au(\d+)-xyz-force\.xyz", l)
a.groups()[0]

'9'

In [4]:
a.groups()[0]

'9'

In [2]:
def conf_count(directory):
    total_conf = 0
    reg = re.compile(r"au(\d+)-xyz-force\.xyz")
    for dirpath, dirnames, filenames in os.walk(directory):
        for filename in filenames:
            result = reg.match(filename)
            if result == None:
                continue
            atnr = int(result.groups()[0])
            with open(dirpath+'/' + filename, 'r') as fp:
                nl = len(fp.readlines())
            conf = nl / (atnr + 2)
            total_conf += conf
#             print atnr, conf
    return total_conf

def  conf_subset(directory, subset_ratio = 0.1, file = 'subset.xyz'):
    reg = re.compile(r"au(\d+)-xyz-force\.xyz")
    for dirpath, dirnames, filenames in os.walk(directory):
        for filename in filenames:
            result = reg.match(filename)
            if result == None:
                continue
            atnr = int(result.groups()[0])
            with open(dirpath+'/' + filename, 'r') as fp:
                lines = fp.readlines()
                nl = len(lines)
            conf = nl / (atnr + 2)
            subset_conf = range(conf)
            np.random.shuffle(subset_conf)
            subset_conf = subset_conf[: int(subset_ratio * conf)]
            subset_conf.sort()
            with open(file, 'a') as fp:
                for i in subset_conf:
                    for j in range(atnr+2):
                        fp.write(lines[i*(atnr+2)+j])

def subset_count(file = 'subset.xyz'):
    conf_energy = {}
    with open(file, 'r') as fp:
        lines = fp.readlines()
    ln = 0
    while ln < len(lines):
        conf = int(lines[ln])
        ln += 1
        energy = float(lines[ln].split()[1])
        if conf_energy.has_key(conf):
            conf_energy[conf].append(energy)
        else:
            conf_energy[conf] = [energy]
        ln += (conf+1)
    return conf_energy


# print conf_count('./')
conf_subset('./', 0.15)
conf_energy = subset_count()
conf_count = {}
for key in conf_energy.keys():
    conf_count[key] = len(conf_energy[key])



30295

In [4]:
reduce(lambda x, y: x+y, conf_count.values())
plt.bar(conf_count.keys(), conf_count.values(), )
plt.show()

In [9]:
plt.hist(conf_energy[12], bins=100)
plt.show()